# 로케이션 할당

### 작업 순서 설명

> (선택)으로 표시되어 있는 부분은 진행하지 않으셔도 됩니다! 상황에 따라 필요하시다면 진행해주시면 됩니다!

0. **모듈 호출**
1. **column 값 확인**
2. **박스 필터링** (선택)
3. **랭크 필터링** (선택)
4. **plt 분리** (선택)
5. **zone 할당**

### 0. 모듈 호출 및 데이터 저장
- 코드를 돌리기 위해 필요한 내용을 호출합니다.
    - 모든 실행은 `shift` + `enter`나 재생 버튼을 눌러주시면됩니다.
- 작업하고자 하시는 데이터 저장은 로케이션 할당 폴터의 `input` 폴더에 진행해주세요
    - path 부분에 파일명을 작성해주시면 됩니다.
    - raw data는 엑셀에서 csv로 추출해서 넣어주시는게 제일 좋습니다.

In [1]:
from location_assignment import *
import pandas as pd
import numpy as np
import random
import sys
import os
import openpyxl
import warnings
warnings.filterwarnings("ignore")

seed = 42
random.seed(seed)

input뒤에 파일명을 넣어주시면 됩니다.
- 경로가 입력되지 않으면, input 파일의 예시 파일(LA_givendata_sample.csv)을 통해 작업합니다.
- 또한, 공백이 없는 편이 안전합니다.

In [2]:
path = "input/LA_givendata_sample.csv"
raw_df = check_df(path)
pre_df = raw_df.copy()
pre_df.head()

,출하일자,주문Code1,Sku Code,상품명,박스당PCS,총수량,가로(mm),세로(mm),높이(mm),오더번호,pcs출하
0,2023.03.07,B2C(택배),W00301,艾多美 洗面乳 1瓶,40,4,143,96,132,723030501912,4
1,2023.03.07,B2C(택배),W00531,艾多美 愛丹 1瓶,54,5,143,96,132,723030501912,5
2,2023.03.07,B2C(택배),W00261,艾多美 BB霜 1瓶,91,2,143,96,132,723030503159,2
3,2023.03.07,B2C(택배),W00276,艾多美 凝萃煥膚防曬乳,70,4,48,23,138,723030503159,4
4,2023.03.07,B2C(택배),W00501,艾多美 牙膏200g(5條)*1組,10,3,143,96,132,723030503159,3


### 1. column 값 확인
- 아래의 코드를 돌리면, 시스템에서 인식한 column 값이 나오게 됩니다.
    - 임의로 내용을 입력해도 좋지만, 아래 코드의 결과물을 복사, 붙여넣기해서 진행하시는게 좋습니다.

In [3]:
# pre_df's columns
pre_df.columns

Index(['출하일자', '주문Code1', 'Sku Code', '상품명', '박스당PCS', '총수량', '가로(mm)',
       '세로(mm)', '높이(mm)', '오더번호', 'pcs출하'],
      dtype='object')

공통 선택 부분입니다.(해당 부분은 무조건 있어야 하는 데이터입니다.)
- 하단의 assert ~ 부분은 column이 있는지 체크하는 코드입니다.
    - 아무런 결과값이 없다면 아래로 넘어가시면 됩니다.
    - AssertionError가 발생했다면, 화살표가 표시된 column 값에 오타가 있는 상황입니다. 수정해주세요
- 하단의 각 값은 아래와 같습니다.
    - zone_num: 존의 개수입니다.
    - trial_row: 크기순 정렬될 상품의 row 수입니다. 예를 들어 zone_num이 10이고, 3이라면 30개가 크기순으로 정렬되고, 이후는 랜덤으로 배치합니다.    
        - 유의미하게 큰 데이터를 생각해 넣어주시는게 좋습니다.
    - error_percentage: 오차범위에 대한 상,하한입니다. 해당하는 비율만큼 오차를 허용합니다.
        - % 변경을 위해서는 100을 곱해서 생각해주세요

In [4]:
order_col_name = "오더번호" #주문번호
assert order_col_name in pre_df.columns

sku_col_name = "Sku Code" #품목코드
assert sku_col_name in pre_df.columns

pcs_col_name = "pcs출하" #작업량
assert pcs_col_name in pre_df.columns

width = 80
zone_num = 10
trial_row = 3
error_percentage = 0.05

2번의 박스 필터링에 필요한 내용입니다. (선택 사항입니다)
- 하단은 기준에 대한 부분을 설정합니다.

In [5]:
pre_b2c_col_name =  "주문Code1" 
assert pre_b2c_col_name in pre_df.columns

width_col_name = "가로(mm)"
assert width_col_name in pre_df.columns

height_col_name = "세로(mm)"
assert height_col_name in pre_df.columns

depth_col_name = "높이(mm)"
assert depth_col_name in pre_df.columns

b2b_min = 55.80
b2c_min = 53.41

3번의 rank 설정을 위해 필요한 내용입니다.
- 한개만 진행시: ["A"]
- 여러개 진행시: ["A", "B"]

In [6]:
rank_col_name = "Rank"
assert rank_col_name in pre_df.columns

filter_rank = ["A"]
assert pre_df[rank_col_name].isin(filter_rank).all()

AssertionError: 

### 2. 박스 필터링
- 위에서 설정한 박스 상한을 넘은 주문은 2개로 분리를 해줍니다. 
    - 결과물은 2개가 존재합니다.
        - 각 주문별로 계산된 값은 output의 `_box_grouped`라는 이름으로 저장됩니다.
        - raw data에 추가된 버전은 output의 `_box_filtered`라는 이름으로 저장됩니다.

In [6]:
b2c_volume_df = update_b2c_volume(pre_df, width, pre_b2c_col_name, width_col_name, height_col_name, depth_col_name, pcs_col_name)
b2c_grouped_df = classify_b2c(b2c_volume_df, width, order_col_name, sku_col_name)
file_name = path.split('/')[-1].split('.')[0] + '_box_grouped.csv'
b2c_grouped_df.to_csv('output/' + file_name, index=False)

box_filtered_df = filtering_box(b2c_grouped_df, b2c_volume_df, width, order_col_name, b2b_min, b2c_min)

file_name = path.split('/')[-1].split('.')[0] + '_box_filtered.csv'
box_filtered_df.to_csv('output/' + file_name, index=False)
pre_df = box_filtered_df.copy()

=================================B2C구분 업데이트 완료==================================
==================================단위부피 업데이트 완료==================================
=================================수량의 부피 업데이트 완료=================================
=================================오더의 박스수 정하기 완료=================================


### 3. Rank Filtering(선택)
- 위에서 설정한 rank 별로 필터링합니다.
    - 해당 부분을 돌린 후에는 Rank에 대해 필터링된 파일을 기준으로 작업하게 됩니다. 
    - 필터링된 값은 `_rank_filtered`라는 이름으로 output folder에 저장됩니다.
- 없는 상태에서 돌리게 되면 에러가 나옵니다.

In [ ]:
file_name = path.split('/')[-1].split('.')[0] + '_rank_filtered.csv'
rank_filtered_df = pre_df[pre_df[rank_col_name].isin(filter_rank)]
rank_filtered_df.to_csv('output/' + file_name, index=False)
pre_df = rank_filtered_df.copy()


### 4. PLT Filtering(선택)
두가지 방법이 존재합니다.
1. 우선 아래에서 정한 PLT 범위에 따라서 시스템에서 자동으로 filtering을 해줍니다.
    - 이때, 30이라면 zone 별로 할당된 작업량의 30%이상에 대해서 필터링한다는 의미입니다.
        - 예시로 10개의 존으로 나눈후, 30이라면 전체의 3%이상의 작업량이 할당되어 있기 때문에, PLT로 빼는 것이 이후 작업량 균등할당을 위해서도 좋기 때문에 해당하는 부분으로 진행되어 있습니다.
    - 이는 `_PLT_filtered`라는 이름으로 output folder에 저장됩니다.
2. csv 파일로 가셔서, 수동으로 설정하실 PLT 그룹을 설정한뒤 하단의 부분을 실행시켜주시면, 수동으로 설정한 데이터를 불러와서 작업하게 됩니다.
    - 꼭 "PLT"로 저장해주세요
    - 단, 시스템에서 돌아가는 PATH 값을 인식하고 있기 때문에, 해당하는 페이지를 끄시게 되면 다 다시 작업하셔야되다보니, 주의해주세요


In [8]:
plt_cut = 31.5

In [9]:
df_grouped = group_order_pcs(pre_df, pcs_col_name, sku_col_name)
df_grouped, df_plt, total_psc, total_order, length = cutting_plt(df_grouped, sku_col_name, plt_cut, zone_num)
df_grouped['zone 할당'] = [0 for _ in range(len(df_grouped))]
# concat df_plt and df_grouped
df_grouped = pd.concat([df_grouped, df_plt], axis=0)
file_name = path.split('/')[-1].split('.')[0] + '_plt_filtered.csv'
df_grouped.to_csv('output/' + file_name, index=False)

### 5. Zone 할당
- 위에서 저장된 `output/{filename}_plt_filtered.csv`를 불러오기를 우선시도하고 진행합니다.

In [10]:
file_name = path.split('/')[-1].split('.')[0] + '_plt_filtered.csv'
pre_zone_df = pd.read_csv('output/' + file_name)
df_grouped = pre_zone_df[pre_zone_df['zone 할당'] == '0']
df_grouped.drop(['zone 할당'], axis=1, inplace=True)
df_plt = pre_zone_df[pre_zone_df['zone 할당'] == "PLT"]

/tmp/ipykernel_4568/2934636788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped.drop(['zone 할당'], axis=1, inplace=True)


In [11]:
df_grouped = zone_assignment(df_grouped, sku_col_name, zone_num, trial_row, error_percentage, total_psc, total_order, length)

zone_df = pd.concat([df_plt, df_grouped], ignore_index=True)
file_name = path.split('/')[-1].split('.')[0] + '_grouped_zone.csv'
zone_df.to_csv('output/' + file_name, index=False)

final_df = merge_zone(pre_df, zone_df, sku_col_name)
final_df.drop(columns=["index"], inplace=True)
file_name = path.split('/')[-1].split('.')[0] + '_final.csv'
final_df.to_csv('output/' + file_name, index=False)

1's exisiting order: 370, exisiting pcs: 488
upper_limit_order: 429.57500000000005, lower_limit_order: 353.42499999999995
upper_limit_sku: 14, lower_limit_sku: 10
selected_indices' sum:  363
2's exisiting order: 377, exisiting pcs: 572
upper_limit_order: 422.57500000000005, lower_limit_order: 346.42499999999995
upper_limit_sku: 14, lower_limit_sku: 10
selected_indices' sum:  363
3's exisiting order: 374, exisiting pcs: 578
upper_limit_order: 425.57500000000005, lower_limit_order: 349.42499999999995
upper_limit_sku: 14, lower_limit_sku: 10
selected_indices' sum:  365
4's exisiting order: 380, exisiting pcs: 530
upper_limit_order: 419.57500000000005, lower_limit_order: 343.42499999999995
upper_limit_sku: 14, lower_limit_sku: 10
selected_indices' sum:  351
5's exisiting order: 380, exisiting pcs: 536
upper_limit_order: 419.57500000000005, lower_limit_order: 343.42499999999995
upper_limit_sku: 14, lower_limit_sku: 10
selected_indices' sum:  371
6's exisiting order: 389, exisiting pcs: 536


/home/skyriver/project/Hokusho_Capstone/final/로케이션 할당/location_assignment.py:303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped["zone 할당"] = zone_list


### 6. 엑셀로 추출
- 최종적으로 엑셀로 추출하는 함수입니다.

In [12]:
file_name = path.split('/')[-1].split('.')[0] + '_box_grouped.csv'
try:
    f_box_grouped_df = pd.read_csv('output/' + file_name)
    print("box_grouped.csv is successfully read.")
except:
    f_box_grouped_df = None
    print("box_grouped.csv is not found.")


box_grouped.csv is successfully read.


In [13]:
# try to read zone_grouped.csv
file_name = path.split('/')[-1].split('.')[0] + '_grouped_zone.csv'
try:
    f_zone_grouped_df = pd.read_csv('output/' + file_name)
    print("grouped_zone.csv is successfully read.")
except:
    f_zone_grouped_df = None
    print("grouped_zone.csv is not found.")

grouped_zone.csv is successfully read.


In [14]:
base_dir = 'output/'
file_name = path.split('/')[-1].split('.')[0] + '_LA.xlsx'
xlxs_dir = os.path.join(base_dir, file_name)

with pd.ExcelWriter(xlxs_dir) as writer:
    final_df.to_excel(writer, sheet_name = '요청자료')
    if f_box_grouped_df is not None:
        f_box_grouped_df.to_excel(writer, sheet_name = '오더의 박스수 정하기')
    if f_zone_grouped_df is not None:
        f_zone_grouped_df.to_excel(writer, sheet_name = '품목_배치zone할당')